In [3]:
options(conflicts.policy = list(warn.conflicts = FALSE))

library(dplyr)
library(ggplot2)
library(latex2exp)
library(gridExtra)

In [5]:
data <- read.csv("../../data/timeseries.csv")
params <- read.csv("../../data/params.csv")

In [6]:
nrows <- 10
nsims <- nrows^2

selected_sims <- 1:nsims

In [ ]:
# Filter the dataset based on the selected simulations and integer range
filtered_data <- data %>%
    filter(sim_number %in% selected_sims, integer %in% 1:10)

In [8]:
# Aggregate data: sum frequency over `chemostat_id`
summarised_data <- filtered_data %>%
  group_by(sim_number, time, integer) %>%
  summarise(frequency = sum(frequency), .groups = "drop")

In [10]:
# Create the plot using facet_wrap
p <- ggplot(summarised_data, aes(x = time, y = frequency, color = factor(integer))) +
    geom_line(alpha = 0.7) +
    
    # Adjust title and axis titles
    labs(title = "Time Series of Simulations", color = "Integer") +
    xlab("Time") +
    ylab("Frequency") +
    
    # Remove the legend, center the title
    theme_minimal() +
    theme(plot.title = element_text(hjust = 0.5), legend.position = "none") +
    
    # Use facet_wrap to create a multi-panel plot
    facet_wrap(~ sim_number, scales = "free_y", labeller = labeller(sim_number = function(x) {
        sim_params <- params %>% filter(sim_number == x)
        sprintf("I=%.2e \n k_d=%.2e", sim_params$inflow_mols, sim_params$outflow_rate)
    })) +
    
    # Annotate each panel with the sim_number
    geom_text(data = summarised_data %>% distinct(sim_number), 
              aes(x = mean(range(summarised_data$time)), y = mean(range(summarised_data$frequency)), label = sim_number), 
              color = "grey", size = 10, hjust = -0.5, vjust = 1.5)

# Save the figure
ggsave("../../figs/multipanel-timeseries.pdf", plot = p, width = 20, height = 20)
